In [1]:
import pandas as pd
import numpy as np


In [2]:
df_m2 = pd.read_csv('D:\study tool\Programming & Data Science Study Materials\Dat Science and AI\Jupyter Notebook files\gofundme_model\model2_final_Data.csv')

In [3]:
df_m2=df_m2.iloc[:,1:]

In [4]:
df_m2['label(%)']=((df_m2['raised_amount($)']/df_m2['days_of_fund_raising'])*60)/df_m2['requested_amount($)']

In [5]:
df_m2['label(%)']

0        233.154000
1          0.000000
2         19.432500
3          1.558413
4         17.241600
            ...    
11995      0.204750
11996      3.195429
11997      0.806481
11998      1.128000
11999      4.243636
Name: label(%), Length: 12000, dtype: float64

In [6]:
df_m2.sample(5)

,vector,requested_amount($),days_of_fund_raising,num_category,raised_amount($),img_context_vector,label(%)
8425,[-0.09101834 0.2510272 -0.38633373 0.041564...,1000.0,34.0,0.051158,600.0,[ 3.4838006 0. 4.320933 0. ...,1.058824
9428,[-0.27083212 0.05386735 -0.24155918 0.280706...,8000.0,86.0,0.027801,1042.0,[ 0. 0. 0. 0. ...,0.090872
3571,[-0.36216494 0.24804871 -0.2465803 0.253902...,5000.0,47.0,0.014079,0.0,[5.7488103 0. 0. 1.432206 0...,0.000000
3784,[-0.35571474 0.3658577 -0.23546869 0.241172...,500.0,83.0,0.014079,0.0,[ 0.13995929 0. 0. 0. ...,0.000000
10446,[-2.78715402e-01 3.49705992e-03 -2.19088033e-...,350.0,92.0,0.016638,550.0,[ 0. 0. 3.168646 2.434044...,1.024845


In [7]:
df_m2.columns

Index(['vector', 'requested_amount($)', 'days_of_fund_raising', 'num_category',
       'raised_amount($)', 'img_context_vector', 'label(%)'],
      dtype='object')

In [8]:
df_m2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   vector                12000 non-null  object 
 1   requested_amount($)   12000 non-null  float64
 2   days_of_fund_raising  12000 non-null  float64
 3   num_category          12000 non-null  float64
 4   raised_amount($)      12000 non-null  float64
 5   img_context_vector    12000 non-null  object 
 6   label(%)              12000 non-null  float64
dtypes: float64(5), object(2)
memory usage: 656.4+ KB


In [9]:
def clear_str(string):
  vector_str = string
  vector_str = vector_str.strip('[]')

  # Split the string into individual number strings
  str_values = vector_str.split()

  # Convert the list of strings to a list of floats
  float_values = [float(num) for num in str_values]
  return float_values

In [10]:
df_label = pd.DataFrame(df_m2['label(%)'])
df_img_context=pd.DataFrame(clear_str(string) for string in df_m2["img_context_vector"])
df_text_context=pd.DataFrame(clear_str(string) for string in df_m2["vector"])
df_amount = df_m2[['requested_amount($)', 'days_of_fund_raising', 'num_category']]

In [11]:
df_img_context.head(1)

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.0,0.0,0.0,6.953934,0.0,0.0,1.672002,0.0,1.960924,0.0,...,0.0,0.0,4.982536,7.310786,0.0,0.0,4.332591,9.158749,2.516675,0.0


In [12]:
df_text_context.head(1)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-0.340189,0.204073,-0.114146,0.377606,-0.244744,0.15282,-0.080407,0.143807,-0.419363,0.271625,...,-0.087962,0.324263,0.058776,-0.586032,-0.332414,-0.340906,0.437036,0.281165,0.155263,0.4137


In [13]:
import tensorflow as tf
from tensorflow import keras
from keras import layers,Input
from keras.models import Model
from keras.layers import Dense,Embedding,LSTM,Flatten,Conv2D,MaxPooling2D,concatenate

Preprocessing & Dimentions reduction:

In [14]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [15]:
text_arr=scaler.fit_transform(df_img_context)
df_img_context=pd.DataFrame(text_arr)
df_img_context.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.642527,-0.181869,-0.605181,2.719069,-0.229247,-0.767336,-0.355620,-0.363319,1.144481,-0.616222,...,-0.493238,-0.849230,0.583441,1.344721,-1.034637,-0.769835,5.384069,2.023554,-0.111655,-0.554200
1,0.415591,-0.181869,-0.605181,-0.447928,-0.229247,-0.767336,-0.431237,-0.363319,-0.372792,-0.616222,...,1.735219,-0.370301,-0.399157,-0.694564,0.468179,-0.769835,-0.238427,-0.699430,-0.266534,-0.554200
2,-0.093033,-0.181869,-0.320797,-0.447928,0.275997,0.730365,0.442200,-0.363319,-0.372792,-0.616222,...,-0.468157,-0.142740,-0.867631,0.505594,-0.536065,0.159920,-0.238427,1.648873,-0.586414,0.276825
3,-0.642527,-0.181869,0.588919,-0.447928,-0.229247,-0.767336,-0.391261,-0.363319,-0.372792,-0.616222,...,-0.493238,-0.849230,1.708784,-0.694564,-0.561849,-0.769835,-0.238427,0.553784,0.021118,-0.554200
4,-0.642527,-0.181869,0.455937,-0.200006,-0.229247,-0.579144,0.266000,-0.363319,-0.372792,2.336324,...,-0.493238,-0.028821,-0.867631,-0.694564,-0.086252,1.140545,-0.238427,-0.072602,-0.872467,-0.554200


In [16]:
text_arr=scaler.fit_transform(df_text_context)
df_text_context=pd.DataFrame(text_arr)
df_text_context.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-0.634810,0.726532,0.823459,1.203708,0.149762,0.136425,-0.211238,-0.194006,-1.178033,-0.396303,...,0.540165,0.093116,1.285075,-2.280538,-1.233187,0.156053,0.307842,1.000326,0.008787,0.942602
1,-0.899957,-0.605034,0.385783,-1.245874,-0.171102,-0.199456,-1.934090,-0.620224,0.416537,-0.632741,...,1.006121,0.174310,0.004309,0.554216,-0.811810,0.702549,0.339486,-0.131980,0.419915,-0.296176
2,1.507654,0.465498,0.512815,2.553884,-0.845310,-1.069210,-1.111144,1.437580,-0.615004,1.642041,...,-0.676799,1.307916,-0.369904,-0.458442,0.450523,0.447241,-0.868356,-0.731681,-0.682661,2.115983
3,0.259333,-0.552576,0.179224,0.510875,-1.612110,-0.461790,-0.421247,-0.224255,-1.158943,-0.414955,...,0.063848,0.802496,-0.497552,-0.895775,-0.426867,0.425690,-0.160362,0.003922,-0.369444,0.660751
4,0.535412,0.157234,-0.717818,1.604144,-0.459423,0.354609,-0.418408,0.800805,-0.020758,0.573064,...,-0.448093,0.858594,-0.128474,0.405743,0.014934,0.615203,-0.864762,-1.808112,0.627411,-0.095619


In [17]:
df_img_context.columns = [f"colm_{i+1}" for i in range(128)]
df_text_context.columns = [f"colm_{i+1}" for i in range(150)]

In [18]:
df_amount.columns

Index(['requested_amount($)', 'days_of_fund_raising', 'num_category'], dtype='object')

In [19]:
arr=scaler.fit_transform(df_amount)
df_amount=pd.DataFrame(arr)
df_amount.columns=['requested_amount($)', 'days_of_fund_raising', 'num_category']
df_amount.head()

,requested_amount($),days_of_fund_raising,num_category
0,-0.025717,-0.939640,2.309283
1,-0.024721,-0.904872,2.309283
2,-0.020393,-0.939640,2.309283
3,-0.024484,-0.279052,2.309283
4,-0.023195,-0.904872,2.309283


In [20]:
input1 = np.array(df_amount)

In [21]:
input1

array([[-0.02571709, -0.93964024,  2.30928318],
       [-0.024721  , -0.90487244,  2.30928318],
       [-0.02039343, -0.93964024,  2.30928318],
       ...,
       [-0.02448424, -0.59196228,  1.05797837],
       [-0.02571709,  0.20769702,  1.05797837],
       [-0.02591322, -0.62673008,  1.05797837]])

In [22]:
from sklearn.decomposition import PCA

pca2 = PCA(n_components=30)
input2 = pca2.fit_transform(df_img_context)
pca3 = PCA(n_components=30)
input3 = pca3.fit_transform(df_text_context)

In [23]:
inputA = Input(shape=(3,))
inputB= Input(shape=(30,))
inputC= Input(shape=(30,))

In [24]:
%pip install keras-tuner --user

Note: you may need to restart the kernel to use updated packages.


In [25]:
df_label.describe()

,label(%)
count,12000.000000
mean,96.709283
std,3362.273178
min,0.000000
25%,0.179700
50%,0.800000
75%,2.921850
max,185168.571429


In [26]:
expected_label=[]
for x in np.array(df_label):
  if x>=100:
    expected_label.append("more than 100 %")
  elif x>=80:
    expected_label.append("between 80 to 100 %")
  elif x>=50:
    expected_label.append("between 50 to 80 %")
  elif x>=20:
    expected_label.append("between 20 to 50 %")
  else:
    expected_label.append("less than 20 %")

In [27]:
category = {
    "more than 100 %": 4,
    "between 80 to 100 %": 3,
    "between 50 to 80 %": 2,
    "between 20 to 50 %":1,
    "less than 20 %":0
}

expected_label = np.array(pd.DataFrame(expected_label).replace(category))
expected_label=expected_label.reshape(-1)

C:\Users\krish\AppData\Local\Temp\ipykernel_20244\1870177616.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  expected_label = np.array(pd.DataFrame(expected_label).replace(category))


In [28]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from keras_tuner import HyperModel
from keras_tuner import RandomSearch

In [41]:
# Define the hypermodel function
def build_model(hp):
    # Define the input layers
    input1 = layers.Input(shape=(3,))
    input2 = layers.Input(shape=(30,))
    input3 = layers.Input(shape=(30,))

    # Define the hyperparameters for the first input branch
    x1 = layers.Dense(units=hp.Int('units_input1', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input1', ['relu', 'tanh', 'sigmoid']))(input1)
    x1 = layers.Dropout(rate=hp.Float('dropout_input1', min_value=0.0, max_value=0.5, step=0.1))(x1)
    x1 = layers.Dense(units=hp.Int('units_input2', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input2', ['relu', 'tanh', 'sigmoid']))(x1)
    x1 = layers.Dropout(rate=hp.Float('dropout_input2', min_value=0.0, max_value=0.5, step=0.1))(x1)
    x1 = layers.Dense(units=hp.Int('units_input3', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input3', ['relu', 'tanh', 'sigmoid']))(x1)
    x1 = layers.Dropout(rate=hp.Float('dropout_input3', min_value=0.0, max_value=0.5, step=0.1))(x1)
    x1 = layers.Dense(units=hp.Int('units_input4', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input4', ['relu', 'tanh', 'sigmoid']))(x1)
    x1 = layers.Dropout(rate=hp.Float('dropout_input4', min_value=0.0, max_value=0.5, step=0.1))(x1)

    # Define the hyperparameters for the second input branch
    y1 = layers.Dense(units=hp.Int('units_input5', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input5', ['relu', 'tanh', 'sigmoid']))(input2)
    y1 = layers.Dropout(rate=hp.Float('dropout_input5', min_value=0.0, max_value=0.5, step=0.1))(y1)
    y1 = layers.Dense(units=hp.Int('units_input6', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input6', ['relu', 'tanh', 'sigmoid']))(input2)
    y1 = layers.Dropout(rate=hp.Float('dropout_input6', min_value=0.0, max_value=0.5, step=0.1))(y1)
    y1 = layers.Dense(units=hp.Int('units_input7', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input7', ['relu', 'tanh', 'sigmoid']))(input2)
    y1 = layers.Dropout(rate=hp.Float('dropout_input7', min_value=0.0, max_value=0.5, step=0.1))(y1)
    y1 = layers.Dense(units=hp.Int('units_input8', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input8', ['relu', 'tanh', 'sigmoid']))(input2)
    y1 = layers.Dropout(rate=hp.Float('dropout_input8', min_value=0.0, max_value=0.5, step=0.1))(y1)

    # Define the hyperparameters for the third input branch
    z1 = layers.Dense(units=hp.Int('units_input9', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input9', ['relu', 'tanh', 'sigmoid']))(input3)
    z1 = layers.Dropout(rate=hp.Float('dropout_input9', min_value=0.0, max_value=0.5, step=0.1))(z1)
    z1 = layers.Dense(units=hp.Int('units_input10', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input10', ['relu', 'tanh', 'sigmoid']))(input3)
    z1 = layers.Dropout(rate=hp.Float('dropout_input10', min_value=0.0, max_value=0.5, step=0.1))(z1)
    z1 = layers.Dense(units=hp.Int('units_input11', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input11', ['relu', 'tanh', 'sigmoid']))(input3)
    z1 = layers.Dropout(rate=hp.Float('dropout_input11', min_value=0.0, max_value=0.5, step=0.1))(z1)
    z1 = layers.Dense(units=hp.Int('units_input12', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_input12', ['relu', 'tanh', 'sigmoid']))(input3)
    z1 = layers.Dropout(rate=hp.Float('dropout_input12', min_value=0.0, max_value=0.5, step=0.1))(z1)

    # Combine the branches
    combined = layers.concatenate([x1, y1, z1])

    # Output layer
    op = layers.Dense(units=hp.Int('units_output1', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_output1', ['relu', 'tanh', 'sigmoid']))(combined)
    op = layers.Dropout(rate=hp.Float('dropout_output1', min_value=0.0, max_value=0.5, step=0.1))(op)
    op = layers.Dense(units=hp.Int('units_output2', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_output2', ['relu', 'tanh', 'sigmoid']))(op)
    op = layers.Dropout(rate=hp.Float('dropout_output2', min_value=0.0, max_value=0.5, step=0.1))(op)
    op = layers.Dense(units=hp.Int('units_output3', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_output3', ['relu', 'tanh', 'sigmoid']))(op)
    op = layers.Dropout(rate=hp.Float('dropout_output3', min_value=0.0, max_value=0.5, step=0.1))(op)
    op = layers.Dense(units=hp.Int('units_output4', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_output4', ['relu', 'tanh', 'sigmoid']))(op)
    op = layers.Dropout(rate=hp.Float('dropout_output4', min_value=0.0, max_value=0.5, step=0.1))(op)
    op = layers.Dense(units=hp.Int('units_output5', min_value=4, max_value=256, step=4), activation=hp.Choice('activation_output5', ['relu', 'tanh', 'sigmoid']))(op)
    op = layers.Dropout(rate=hp.Float('dropout_output5', min_value=0.0, max_value=0.5, step=0.1))(op)
    output = layers.Dense(5, activation='softmax')(op)

    # Define the model
    model = Model(inputs=[input1, input2, input3], outputs=output)

    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd', 'nadam'])

    # Compile the model with the selected optimizer and a tunable learning rate
    if optimizer == 'adam':
      opt = tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'))
    elif optimizer == 'rmsprop':
      opt = tf.keras.optimizers.RMSprop(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'))
    elif optimizer == 'sgd':
      opt = tf.keras.optimizers.SGD(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'))
    elif optimizer == 'nadam':
      opt = tf.keras.optimizers.Nadam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'))

    # Compile the model with a tunable learning rate
    model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

    return model

In [45]:
# Initialize the tuner
tuner = RandomSearch(
    build_model,  # The hypermodel function
    objective='val_loss',  # The objective to minimize
    max_trials=10,  # Number of different models to try
    executions_per_trial=2,  # Number of times to train each model
    directory='my_dir8',  # Where to save the search results
    project_name='tuning'  # Name of the project
)

In [47]:
tuner.search([input1, input2, input3], expected_label,
             validation_split=0.2,
             epochs=5,
             batch_size=tuner.oracle.hyperparameters.Int('batch_size', min_value=4, max_value=128, step=4)
            )

Trial 10 Complete [00h 00m 35s]
val_loss: 0.07505838200449944

Best val_loss So Far: 0.06359228119254112
Total elapsed time: 00h 10m 49s


In [48]:
# Get the optimal hyperparameters (no num_models argument)
best_hyperparameters = tuner.get_best_hyperparameters()[0]

# Get the best model (using num_models argument here)
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
best_model.evaluate([input1, input2, input3], expected_label)

C:\Users\krish\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 51 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9634 - loss: 0.1370


[0.07844755798578262, 0.981333315372467]

In [33]:
import matplotlib.pyplot as plt

In [34]:
plt.plot(history.history["accuracy"],c="red")
plt.plot(history.history["val_accuracy"],c="blue")

NameError: name 'history' is not defined

A->PurPose
B->Story
C->Image (obj -> if sentiment can be found out or like supporting the purpose category(concat with purpose logits in this particular case))



In [ ]:
inputA = Input(shape=(10,))
inputB= Input(shape=(2370,)) #for image
inputC= Input(shape=(28,28,1))

In [ ]:
# layer architechture for A
dlA1=Dense(64,activation='relu')(inputA)
dlA2=Dense(32,activation='relu')(dlA1)
dlA3=Dense(8,activation='relu')(dlA2)

In [ ]:
# layer architechture for B
vocab_size=2370
embedding_dim=250
elB1=Embedding(input_dim=vocab_size,output_dim=embedding_dim)(inputB)
lstmlB2=LSTM(32,return_sequences=True,activation='relu')(elB1)
lstmlB3=LSTM(32,return_sequences=True,activation='relu')(lstmlB2)
lstmlB4=LSTM(8,return_sequences=False,activation='relu')(lstmlB3)
dlB5=Dense(256,activation='relu')(lstmlB4)
dlB6=Dense(124,activation='relu')(dlB5)
dlB7=Dense(64,activation='relu')(dlB6)

In [ ]:
# layer architechture for C
y=Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding="valid", activation='relu')(inputC)
y1=MaxPooling2D(pool_size=(2,2),strides=2,padding='valid')(y)
y2=Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu')(y1)
y3=MaxPooling2D(pool_size=(2,2),strides=2,padding='valid')(y2)
y4=Flatten()(y3)
y5=Dense(128,activation='relu')(y4)
y6=Dense(10,activation='softmax')(y5)

In [ ]:
combined = concatenate([dlA3, y6, dlB7])

In [ ]:
# apply a FC layer and then a regression prediction on the
# combined outputs
op1 = Dense(2, activation="relu")(combined)
op2 = Dense(1, activation="linear")(op1)

In [ ]:
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[inputA, inputB, inputC], outputs=op2)

In [ ]:
import pydot
from tensorflow.keras.utils import plot_model

# Your model code here
plot_model(model)
